# RAG ಬಳಸಿ Azure AI ಏಜೆಂಟ್ ಸೇವೆ ಮತ್ತು ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್

ಈ ಕೋಡ್ ಉದಾಹರಣೆ `Azure AI Agent Service` ಮತ್ತು `Semantic Kernel` ಬಳಸಿ ರಿಟ್ರೀವಲ್-ಆಗ್ಮೆಂಟೆಡ್ ಜನರೇಷನ್ (RAG)ಗಾಗಿ Azure AI ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸಲು ಮತ್ತು ನಿರ್ವಹಿಸಲು ಹೇಗೆ ಸಾಧ್ಯವಿದೆ ಎಂಬುದನ್ನು ತೋರಿಸುತ್ತದೆ. ಈ ಏಜೆಂಟ್ ಬಳಕೆದಾರರ ಪ್ರಶ್ನೆಗಳನ್ನು ಹಿಂಪಡೆಯಲಾದ ಸಂದರ್ಭದ ಆಧಾರದ ಮೇಲೆ ಪ್ರಕ್ರಿಯೆಗೊಳಿಸಿ, ತಕ್ಕ ಉತ್ತರಗಳನ್ನು ಒದಗಿಸುತ್ತದೆ.


## ಪರಿಸರವನ್ನು ಪ್ರಾರಂಭಿಸಲಾಗುತ್ತಿದೆ


SQLite ಆವೃತ್ತಿ ದೋಷ ಸರಿಪಡನೆ  
ನೀವು ಈ ದೋಷವನ್ನು ಎದುರಿಸಿದರೆ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
ನಿಮ್ಮ ನೋಟುಬುಕ್‌ನ ಪ್ರಾರಂಭದಲ್ಲಿ ಈ ಕೋಡ್ ಬ್ಲಾಕ್ ಅನ್ನು ಅನ್ಕಾಮೆಂಟ್ ಮಾಡಿ:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದು ಮಾಡುವುದು
ಕೆಳಗಿನ ಕೋಡ್ ಅಗತ್ಯವಿರುವ ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದು ಮಾಡುತ್ತದೆ:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# ಸೆಮ್ಯಾಂಟಿಕ್ ಕೆರ್ಣಲ್ ಮತ್ತು ಅಜೂರ್ AI ಏಜೆಂಟ್ ಸೇವೆಯೊಂದಿಗೆ Retrieval-Augmented Generation

ಈ ಮಾದರಿ **ಅಜೂರ್ AI ಏಜೆಂಟ್ ಸೇವೆ** ಬಳಸಿ **Retrieval-Augmented Generation (RAG)** ಅನ್ನು ಹೇಗೆ ಮಾಡಬಹುದು ಎಂಬುದನ್ನು ತೋರಿಸುತ್ತದೆ, ಇದು ಭಾಷಾ ಮಾದರಿಯನ್ನು ಅಪ್ಲೋಡ್ ಮಾಡಿದ ಡಾಕ್ಯುಮೆಂಟ್‌ನ ಡೊಮೈನ್-ನಿರ್ದಿಷ್ಟ ವಿಷಯದೊಂದಿಗೆ ಸಂಯೋಜಿಸುತ್ತದೆ.

### ಇದು ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ

1. **ಡಾಕ್ಯುಮೆಂಟ್ ಅಪ್ಲೋಡ್**: ಮಾರ್ಕ್‌ಡೌನ್ ಫೈಲ್ (document.md) ಅನ್ನು ಏಜೆಂಟ್ ಸೇವೆಗೆ ಅಪ್ಲೋಡ್ ಮಾಡಲಾಗುತ್ತದೆ, ಇದರಲ್ಲಿ (ಕಾಂಟೋಸೋನ ಪ್ರಯಾಣ ವಿಮಾ ನೀತಿ) ಮಾಹಿತಿ ಇದೆ.

2. **ವೆಕ್ಟರ್ ಸ್ಟೋರ್ ರಚನೆ**: ಡಾಕ್ಯುಮೆಂಟ್ ಅನ್ನು ವೆಕ್ಟರ್ ಸ್ಟೋರ್‌ಗೆ ಸೂಚ್ಯಂಕಗೊಳಿಸಲಾಗುತ್ತದೆ, ಇದರ ವಿಷಯದ ಮೇಲೆ ಸೆಮ್ಯಾಂಟಿಕ್ ಶೋಧವನ್ನು ಸಕ್ರಿಯಗೊಳಿಸಲು.

3. **ಏಜೆಂಟ್ ಕಾನ್ಫಿಗರೇಶನ್**: `gpt-4o` ಮಾದರಿಯನ್ನು ಬಳಸಿಕೊಂಡು ಏಜೆಂಟ್ ಅನ್ನು ಈ ಕೆಳಗಿನ ಕಠಿಣ ಸೂಚನೆಗಳೊಂದಿಗೆ ಸ್ಥಾಪಿಸಲಾಗುತ್ತದೆ:
   - ಡಾಕ್ಯುಮೆಂಟ್‌ನಿಂದ ಹಿಂಪಡೆಯಲಾದ ವಿಷಯದ ಆಧಾರದ ಮೇಲೆ ಮಾತ್ರ ಪ್ರಶ್ನೆಗಳಿಗೆ ಉತ್ತರಿಸಬೇಕು.
   - ಪ್ರಶ್ನೆ ವ್ಯಾಪ್ತಿಯ ಹೊರಗಿದ್ದರೆ ಉತ್ತರಿಸಲು ನಿರಾಕರಿಸಬೇಕು.

4. **ಫೈಲ್ ಶೋಧ ಸಾಧನ ಸಮೀಕರಣ**: `FileSearchTool` ಅನ್ನು ಏಜೆಂಟ್‌ನೊಂದಿಗೆ ನೋಂದಾಯಿಸಲಾಗುತ್ತದೆ, ಇದು ಮಾದರಿಯನ್ನು ಶೋಧಿಸಲು ಮತ್ತು ನಿರ್ವಚನೆಯ ಸಮಯದಲ್ಲಿ ಸೂಚ್ಯಂಕಗೊಳಿಸಿದ ಡಾಕ್ಯುಮೆಂಟ್‌ನಿಂದ ಸಂಬಂಧಿತ ಭಾಗಗಳನ್ನು ಹಿಂಪಡೆಯಲು ಸಕ್ರಿಯಗೊಳಿಸುತ್ತದೆ.

5. **ಬಳಕೆದಾರರ ಸಂವಹನ**: ಬಳಕೆದಾರರು ಪ್ರಶ್ನೆಗಳನ್ನು ಕೇಳಬಹುದು. ಡಾಕ್ಯುಮೆಂಟ್‌ನಲ್ಲಿ ಸಂಬಂಧಿತ ಮಾಹಿತಿ ಕಂಡುಬಂದರೆ, ಏಜೆಂಟ್ ಆಧಾರಿತ ಉತ್ತರವನ್ನು ರಚಿಸುತ್ತದೆ.  
   ಮಾಹಿತಿ ಸಾಕಷ್ಟು ಲಭ್ಯವಿಲ್ಲದಿದ್ದರೆ, ಡಾಕ್ಯುಮೆಂಟ್ದಲ್ಲಿ ಅಗತ್ಯ ಮಾಹಿತಿ ಇಲ್ಲ ಎಂದು ಸ್ಪಷ್ಟವಾಗಿ ಉತ್ತರಿಸುತ್ತದೆ.


### ಮುಖ್ಯ ಕಾರ್ಯಕ್ಷಮತೆ


ನೀವು `DefaultAzureCredential` ಬಳಸುವಾಗ ಸರಿಯಾದ ಪ್ರಾಮಾಣೀಕರಣ ಸಂದರ್ಭವನ್ನು ಒದಗಿಸಲು ದಯವಿಟ್ಟು ಮೊದಲು Azure CLI ಬಳಸಿ `az login` ಅನ್ನು ಚಲಾಯಿಸಿ. Azure AI ಏಜೆಂಟ್ ಸೇವೆ API ಕೀಗಳನ್ನು ಬಳಸುವುದಿಲ್ಲ.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

ನೀವು ಈ ಕೆಳಗಿನಂತೆ ಔಟ್‌ಪುಟ್ ಅನ್ನು ನೋಡಬಹುದು:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಾಖಲೆ AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಖಚಿತತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅರ್ಥೈಸುವಿಕೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
